# GCC118 - Programação Matemática  
## Universidade Federal de Lavras  

---

## Problema 1  

### Alunos:  
- **Gabriel Jardim de Souza** - 202310530  
- **Paulo Henrique Silveira dos Anjos** - 202310533


## Enunciado:

A Empresa de Viação Aérea Brasileira está estudando a compra de três tipos de aviões:  
- Boeing 717 para as pontes aéreas a curta distância,  
- Boeing 737-500 para vôos domésticos e internacionais de média distância,  
- MD-11 para vôos internacionais de longa distância.  

Em um estudo preliminar, considerou-se que a capacidade máxima dos aviões a serem comprados será sempre preenchida para efeito de planejamento. Os dados de planejamento constam na Tabela 1.  

---


**Tabela 1: Informações sobre os aviões**


| Avião                 | Custo (Milhões) | Receita (Milhões) | Pilotos aptos |
|-----------------------|-----------------|-------------------|---------------|
| **Boeing 717**        | 5,1             | 330               | 30            |
| **Boeing 737-500**    | 3,6             | 300               | 20            |
| **MD-11**             | 6,8             | 420               | 10            |


---

A verba disponível para as compras é de **220 milhões de dólares**. Os pilotos de MD-11 podem pilotar todos os aviões da empresa, mas os demais pilotos só podem ser escalados às aeronaves a que foram habilitados. Cada aeronave necessita de **dois pilotos** para operar. As oficinas de manutenção podem suportar até **40 Boeings 717**. Um Boeing 737-500 equivale, em esforço de manutenção, a **3/4**, e um MD-11 a **5/3**, quando referidos ao Boeing 717.   

---

### Objetivo

Apresente um modelo que otimize as aquisições de aviões nesta empresa, **maximizando as receitas**.


## Instalação da biblioteca PuLP

In [ ]:
!pip install pulp
import pulp

## Declaração dos parâmetros

* $aviao$: conjunto de tipos de aviões disponíveis (0, 1, 2).  
* $custo$: lista com o custo de aquisição de cada tipo de avião.  
* $receita$: lista com a receita obtida por cada tipo de avião.  
* $pilotos\_aptos$: lista com o número de pilotos disponíveis para cada tipo de avião.  
* $pesos\_de\_manutencao$: lista com o peso relativo de manutenção de cada tipo de avião.  



In [ ]:
aviao = range(3)

custo = [5.1, 3.6, 6.8]

receita = [330, 300, 420]

pilotos_aptos = [30, 20, 10]

pesos_de_manutencao = [1, 3/4, 5/3]

## Declaração do objeto que representa o modelo matemático

In [ ]:
modelo = pulp.LpProblem('problema1', pulp.LpMaximize)

## Variáveis de decisão

* $avioes\_adquiridos[i] \ge 0$: número de aviões do tipo $i$ a serem adquiridos, $i \in aviao$.  
* $pilotos\_md[i] \ge 0$: número de pilotos MD-11 alocados para o tipo de avião $i$, $i \in aviao$.  


In [ ]:
avioes_adquiridos = pulp.LpVariable.dicts('avioes', aviao, lowBound=0, cat='Integer')
pilotos_md = pulp.LpVariable.dicts('pilotos_md', aviao, lowBound=0, cat='Integer')

## Função objetivo

* Maximização da receita total: $\max \sum_{i \in aviao} receita[i] \cdot avioes\_adquiridos[i]$.  


In [ ]:
modelo += pulp.lpSum(receita[i] * avioes_adquiridos[i] for i in aviao)

## Restrições

* Restrição de orçamento: $\sum_{i \in aviao} custo[i] \cdot avioes\_adquiridos[i] \le 220$.  

In [ ]:
modelo += pulp.lpSum(custo[i]*avioes_adquiridos[i] for i in aviao) <= 220

* Restrição de pilotos para o avião do tipo **Boeing 717**: $2 \cdot avioes\_adquiridos[0] \le pilotos\_aptos[0] + pilotos\_md[0]$.  


In [ ]:
modelo += (2*avioes_adquiridos[0]) <= pilotos_aptos[0] + pilotos_md[0]

* Restrição de pilotos para o avião do tipo **Boeing 737-500**: $2 \cdot avioes\_adquiridos[1] \le pilotos\_aptos[1] + pilotos\_md[1]$.  

In [ ]:
modelo += (2*avioes_adquiridos[1]) <= pilotos_aptos[1] + pilotos_md[1]

* Restrição de pilotos para o avião do tipo **MD-11**: $2 \cdot avioes\_adquiridos[2] \le pilotos\_aptos[2] - pilotos\_md[0] - pilotos\_md[1]$.  

In [ ]:
modelo += (2*avioes_adquiridos[2]) <= pilotos_aptos[2] - pilotos_md[0] - pilotos_md[1]

* Restrição de capacidade das oficinas de manutenção: $\sum_{i \in aviao} pesos\_de\_manutencao[i] \cdot avioes\_adquiridos[i] \le 40$.  

In [ ]:
modelo += pulp.lpSum(pesos_de_manutencao[i]*avioes_adquiridos[i] for i in aviao) <= 40

### Resolvendo o problema

In [ ]:
status = modelo.solve()

## Imprimindo as soluções do problema

In [ ]:
descricao_avioes = {
    0: "Boeing 717",
    1: "Boeing 737-500",
    2: "MD-11"
}

print('STATUS:', pulp.LpStatus[status])
print('FUNÇÃO OBJETIVO: $', modelo.objective.value(), 'milhões')
print('\nSOLUÇÕES:')

for a in aviao:
    print(f"Tipo: {descricao_avioes[a]}  Aviões adquiridos: {avioes_adquiridos[a].value()}")


STATUS: Optimal
FUNÇÃO OBJETIVO: $ 10050.0 milhões

SOLUÇÕES:
Tipo: Boeing 717  Aviões adquiridos: 15.0
Tipo: Boeing 737-500  Aviões adquiridos: 10.0
Tipo: MD-11  Aviões adquiridos: 5.0
